In [ ]:
import pandas as pd
import numpy as np
from datetime import datetime
import os
import pickle
import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
# raw df, 46000 songs
with open('grand_frame.pkl', 'rb') as readfile:
    grand_frame = pickle.load(readfile)

## Frame reduction

In [ ]:
# column for main artist, ignoring featured artists
grand_frame['primary'] = grand_frame['artist'].apply(lambda x: x[0])

In [ ]:
# create separate df with value counts
counts_df = grand_frame['primary'].value_counts()

value_frame = pd.DataFrame({ 'primary': counts_df.index.tolist(), 
                             'counts': counts_df.values.tolist() })

In [ ]:
# merge two dfs to add 'primary artist' and 'value counts' columns
final_frame = pd.merge(grand_frame,value_frame, on='primary')

In [ ]:
# drop songs with artists who do not appear at least 10 times in df
ten_frame = final_frame[final_frame['counts'] > 10]

In [ ]:
##with open('ten_frame.pkl', 'wb') as writefile:
##    pickle.dump(ten_frame, writefile)

#### Vectorize

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer

In [ ]:
Tvectorizer = TfidfVectorizer(analyzer=text_process, min_df=.01, max_df=1000, max_features=10000)

In [ ]:
tfidf = Tvectorizer.fit_transform(ten_frame['lyrics'])

In [ ]:
##with open('tfidf.pkl', 'wb') as writefile:
##    pickle.dump(tfidf, writefile)

#### PCA

In [ ]:
from sklearn.decomposition import PCA
pca = PCA(n_components=2)
prin_var = pca.fit(tfidf.toarray()) 
principal = pca.fit_transform(tfidf.toarray())

In [ ]:
pvars = prin_var.explained_variance_

In [ ]:
plt.plot(range(1,len(pvars)+1),pvars)
plt.xlabel('Principal Components')
plt.ylabel('Explained Variance')
plt.title('Variance by Component Number')

#### KMEANS

In [ ]:
from sklearn.cluster import KMeans

In [ ]:
# determine number of clusters
n_inertia = []
for i in range(1,15):
    km = KMeans(n_clusters = i, init = 'k-means++', n_init=25, n_jobs=-1)
    kmeans_pca_array = km.fit_transform(principal)
    pca_clusters_array = km.predict(principal)
    inertia = km.inertia_
    n_inertia.append(inertia)

In [ ]:
plt.plot(range(1,15),n_inertia)